In [1]:
'''
For prior assumpsion, we take the medically known values and some rough assumptions on our behalf-
For y=0
    Age - 30 (Younger people usually have higher tendency)
    Rest Blood Pressure - 120 (According to my understanding of what I obtained from the internet)
    Cholestrol Level - 162.5 (125 to 200 - we bluntly take the average, therefore, = 162.5)
    
For y=1
    Age - 60 
    Rest Blood Pressure - 150 
    Cholestrol Level - 200 

Variance we randomly take as 1 across all features and covariance as 0 (We will vary it and see the results)
'''

'\nFor prior assumpsion, we take the medically known values and some rough assumptions on our behalf-\nFor y=0\n    Age - 30 (Younger people usually have higher tendency)\n    Rest Blood Pressure - 120 (According to my understanding of what I obtained from the internet)\n    Cholestrol Level - 162.5 (125 to 200 - we bluntly take the average, therefore, = 162.5)\n    \nFor y=1\n    Age - 60 \n    Rest Blood Pressure - 150 \n    Cholestrol Level - 200 \n\nVariance we randomly take as 1 across all features and covariance as 0 (We will vary it and see the results)\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import math
import re

In [3]:
def import_dataset(split_percent = 70):
    dataset = pd.read_csv('E:\ELL_project\problem1\health_data.csv')
    dataset = dataset.sample(frac = 1)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1:].values
    datasize = X.shape[0]

    split_point = split_percent//10

    X_train = X[:(datasize*split_point)//10,:]
    y_train = y[:(datasize*split_point)//10,:]
    X_test = X[(datasize*split_point)//10:,:]
    y_test = y[(datasize*split_point)//10:,:]

    return(X_train,X_test,y_train,y_test)


In [4]:
def feature_scaling(X_train,X_test):
    X_mean = np.sum(X_train,axis=0)
    X_var = np.sqrt(np.sum(np.square(X_train - X_mean), axis=0))

    X_train_feat_scaled = (X_train - X_mean ) / X_var
    X_test_feat_scaled = (X_test - X_mean) / X_var

    return (X_mean,X_var, X_train_feat_scaled,X_test_feat_scaled)

In [5]:
def sep_01(X_train,y_train):
    x_0_train = X_train[y_train[:,0]==0]
    x_1_train = X_train[y_train[:,0]==1]
    return(x_0_train,x_1_train)

In [6]:
def mean_var(x):
    siz = x.shape[0]
    feat = x.shape[1]
    mean = np.sum(x,axis=0)/siz
    mean = mean.reshape((1,feat))
    dif = x-mean
    covariance_matrix = np.dot(np.transpose(dif),dif) / siz

    return(mean,covariance_matrix)

In [7]:
def gen_meu_var(m1=30,m2=120,m3=162.5,v1=1,v2=1,v3=1,m11=60,m12=150,m13=200,v11=1,v12=1,v13=1):
    num_feat = 3
    prior_mean_0 = np.array([m1,m2,m3])
    prior_variance_0 = np.zeros((num_feat,num_feat))
    prior_variance_0[0,0] = v1
    prior_variance_0[1,1] = v2
    prior_variance_0[2,2] = v3

    # prior_variance_0 = np.array([v1,v2,v3])

    prior_mean_1 = np.array([m11,m12,m13])
    prior_variance_1 = np.zeros((num_feat,num_feat))
    prior_variance_1[0,0] = v11
    prior_variance_1[1,1] = v12
    prior_variance_1[2,2] = v13
    # prior_variance_1 = np.array([v11,v12,v13])

    prior_mean_0 = prior_mean_0.reshape((1,num_feat))
    prior_mean_1 = prior_mean_1.reshape((1,num_feat))

    # print('prior_meu_shape_0: ', prior_mean_0.shape)
    # print('prior_meu_shape_1: ', prior_mean_1.shape)

    prior_vals = [0,0,0,0]
    prior_vals = [prior_mean_0,prior_variance_0,prior_mean_1,prior_variance_1]
    return(prior_vals)

In [8]:
def ret_n(meu0,meu,cov0,cov,data_size):
    # Now, we will consider L as the left most value, M as the middle one and R as the right one in the formula for meu        # and cov matrix
    l = cov0
    m = np.linalg.inv(cov0 + (cov/data_size))
    r = cov/data_size

    meu_n = np.dot(np.dot(l,m),np.transpose(meu)) + np.dot(np.dot(r,m),np.transpose(meu0))
    meu_n = np.transpose(meu_n)
    # print('sfsgs',meu_n)
    cov_n = np.dot(np.dot(l,m),r)

    return (meu_n,cov_n)

In [9]:
def mean_var_data(X_train,y_train,prior_vals):
    x_0,x_1 = sep_01(X_train,y_train)
    meu_0,covar_0 = mean_var(x_0)
    meu_1,covar_1 = mean_var(x_1)
    # print(meu_1,'verbe')

    prior_mean_var = prior_vals
    
    prior_mean_0 = prior_mean_var[0]
    prior_covar_0 = prior_mean_var[1]
    prior_mean_1 = prior_mean_var[2]
    prior_covar_1 = prior_mean_var[3]

    size_0 = x_0.shape[0]
    meu_n_0,cov_n_0 = ret_n(prior_mean_0,meu_0,prior_covar_0,covar_0,size_0)
    size_1 = x_1.shape[0]
    meu_n_1,cov_n_1 = ret_n(prior_mean_1,meu_1,prior_covar_1,covar_1,size_1)

    return(meu_n_0,cov_n_0, meu_n_1,cov_n_1)


In [10]:
def eval_on_test(X_test,meu_n_0,cov_n_0,meu_n_1,cov_n_1):

    num_of_feat = cov_n_0.shape[0]
    det_0 = np.linalg.det(cov_n_0)
    det_1 = np.linalg.det(cov_n_1)

    p0 = 1/math.pow((2*math.pi),num_of_feat/2)
    p0 = p0 / (det_0**0.5)
    p1 = 1/math.pow((2*math.pi),num_of_feat/2)
    p1 = p1 / (det_1**0.5)

    m0 = np.transpose(meu_n_0)
    m1 = np.transpose(meu_n_1)

    print('m0: ',m0.shape)

    exp_0 = np.exp(-0.5 * (np.dot(X_test-meu_n_0,np.linalg.inv(cov_n_0)))*(X_test-meu_n_0))
    exp_1 = np.exp(-0.5 * (np.dot(X_test-meu_n_1,np.linalg.inv(cov_n_1)))*(X_test-meu_n_1))

    # exp_0 = np.exp(-0.5 * np.sum(np.multiply(np.dot(X_test-meu_n_0,np.linalg.inv(cov_n_0)),(X_test-meu_n_0)),axis=1).reshape(-1,1))
    # exp_1 = np.exp(-0.5 * np.sum(np.multiply(np.dot(X_test-meu_n_1,np.linalg.inv(cov_n_1)),(X_test-meu_n_1)),axis=1).reshape(-1,1))

    print(exp_0.shape)

    # exp_0 = np.exp(-0.5 * (np.dot(np.dot(X_test-m0,np.linalg.inv(cov_n_0)),np.transpose(X_test-m0))))
    # exp_1 = np.exp(-0.5 * (np.dot(np.dot(X_test-m1,np.linalg.inv(cov_n_1)),np.transpose(X_test-m1))))

    # print('Xtest: ',X_test.shape)
    # print('meu_n_0: ', meu_n_0.shape)
    # print('cov_n_0: ', cov_n_0.shape)
    # print('meu_n_1: ', meu_n_1.shape)
    # print('cov_n_1: ', cov_n_1.shape)
    
    # print(exp_0.shape)

    return ( exp_0/p0 , exp_1/p1 )

In [11]:
def predicter(prob_X_0,prob_X_1,threshold=1):
    y_pred = np.zeros(prob_X_0.shape)
    y_pred[prob_X_1[:,:]/prob_X_0[:,:] > threshold] = 1

    return y_pred

In [12]:
def accuracy_metrics(y_pred_thresh, y_test):

    test_size = y_test.shape[0]

    tp = np.sum((y_pred_thresh+y_test)==2 , axis=0)[0]
    tn = np.sum(y_pred_thresh==y_test , axis=0)[0] - tp
    fp = np.sum(y_pred_thresh , axis=0)[0]-tp
    fn = test_size-tp-tn-fp


    print('tp: {} , tn: {} , fp: {} , fn: {}'.format(tp,tn,fp,fn))

    acc = (tp+tn)/test_size
    prec = (tp)/(tp+fp)
    recl = (tp)/(tp+fn)
    f1 = 2*prec*recl/(prec+recl)

    print('Accuracy: {}'.format( acc  ))
    print('Precision: {}'.format( prec  ))
    print('Recall: {}'.format( recl  ))
    print('F1 score: {}'.format( f1  ))

In [13]:
def map_estimator(threshold=1,split_percent=70):
    X_train,X_test,y_train,y_test = import_dataset(split_percent)
    X_train_0 , X_train_1 = sep_01(X_train,y_train)
    prior_vals = gen_meu_var(m1=30,m2=120,m3=162.5,v1=1,v2=1,v3=1,m11=60,m12=150,m13=200,v11=1,v12=1,v13=1)
    meu_n_0,cov_n_0, meu_n_1,cov_n_1 = mean_var_data(X_train,y_train,prior_vals)
    # print(meu_n_0)
    prob_0,prob_1 = eval_on_test(X_test,meu_n_0,cov_n_0,meu_n_1,cov_n_1)
    print(prob_0.shape,prob_1.shape)
    y_pred = predicter(prob_0,prob_1,threshold)
    accuracy_metrics(y_pred, y_test)

In [84]:
map_estimator(threshold = 1)

m0:  (3, 1)
(210, 3)
(210, 3) (210, 3)
tp: 73 , tn: 94 , fp: 25.0 , fn: 18.0
Accuracy: 0.7952380952380952
Precision: 0.7448979591836735
Recall: 0.8021978021978022
F1 score: 0.7724867724867726
